# Argus‑style Possibility Curves for MATIF Rapeseed – Research Notebook  🛠️📈

This notebook walks through **end‑to‑end construction, training and evaluation** of a 4‑parameter skew‑t Possibility‑Curve model:

1. **Concept primer** – what APCs are and why skew‑t.
2. **Data prep** – rolling front‑month futures & feature engineering helpers.
3. **Model code** – TensorFlow Probability network predicting (μ, σ, ν, τ).
4. **Training loop** – external Python loop for full transparency & logging.
5. **Diagnostics** – quantile bands, PIT histogram, skew/tail dynamics.

> 💡  *Every code block is followed by dual comments:* business reasoning and implementation insights.

🟡 **This notebook does in‑sample exploration only**. Out‑of‑sample scoring comes next.

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tfd, tfb = tfp.distributions, tfp.bijectors

## 1️⃣  Data Preparation

In [ ]:
# y = target, X = features (from df_4_params)
y_raw = df_4_params["close_rolled"].values.astype("float32")
X_raw = df_4_params[["sma_20", "sma_50", "rsi_14", "atr_14", "bb_pct", "z_score"]].values.astype("float32")
y_mean, y_std = y_raw.mean(), y_raw.std()
y_z = ((y_raw - y_mean) / y_std).reshape(-1, 1)
norm = tf.keras.layers.Normalization(); norm.adapt(X_raw)

## 2️⃣  Model Definition – μ, σ, ν, τ network

In [ ]:
class APCSkewTModel(tf.keras.Model):
    def __init__(self, hidden=32):
        super().__init__()
        self.norm = norm
        self.d1 = tf.keras.layers.Dense(hidden, activation="relu")
        self.d2 = tf.keras.layers.Dense(hidden, activation="relu")
        self.mu = tf.keras.layers.Dense(1)
        self.log_s = tf.keras.layers.Dense(1, bias_initializer=tf.constant_initializer(np.log(1.)))
        self.skew_h = tf.keras.layers.Dense(1)
        self.log_t = tf.keras.layers.Dense(1, bias_initializer=tf.constant_initializer(np.log(1.)))

    def call(self, x):
        x = self.norm(x)
        h = self.d2(self.d1(x))
        mu = self.mu(h)
        sigma = tf.exp(self.log_s(h))
        skew = 5.0 * tf.tanh(self.skew_h(h))
        tail = tf.exp(self.log_t(h))
        return mu, sigma, skew, tail

In [ ]:
def make_dist(mu, sigma, skew, tail):
    base = tfd.StudentT(df=3., loc=0., scale=1.)
    return tfd.TransformedDistribution(base, tfb.Chain([
        tfb.Shift(mu), tfb.Scale(sigma),
        tfb.SinhArcsinh(skewness=skew, tailweight=tail)
    ]))

## 3️⃣  Training Loop

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_raw, y_z)).shuffle(500).batch(64)
model = APCSkewTModel()
opt = tf.keras.optimizers.Adam(1e-3)

@tf.function
def step(xb, yb):
    with tf.GradientTape() as tape:
        mu, s, k, t = model(xb)
        loss = -tf.reduce_mean(make_dist(mu, s, k, t).log_prob(yb))
    g,_ = tf.clip_by_global_norm(tape.gradient(loss, model.trainable_variables), 2.)
    opt.apply_gradients(zip(g, model.trainable_variables))
    return loss

loss_log = []
for ep in range(30):
    loss_ep = np.mean([step(bx, by).numpy() for bx, by in train_ds])
    loss_log.append(loss_ep)
    print(f"Epoch {ep+1:02d}: NLL {loss_ep:.4f}")

## 4️⃣  In‑Sample Diagnostics

In [ ]:
mu, s, k, t = model(X_raw)
dist = make_dist(mu, s, k, t)
q = tf.stack([dist.quantile(p) for p in [0.1,0.25,0.5,0.75,0.9]], axis=-1).numpy()
q_price = q * y_std + y_mean

plt.figure(figsize=(10,5))
plt.plot(df_4_params["date"], y_raw, label="Actual", lw=1)
for i,label in enumerate(["P10","P25","P50","P75","P90"]):
    plt.plot(df_4_params["date"], q_price[:,i], ls="--", label=label)
plt.legend(); plt.title("In‑sample Quantile Bands"); plt.grid(True); plt.show()

pit = dist.cdf(y_z.flatten()).numpy()
plt.hist(pit, bins=20, edgecolor='k'); plt.title("PIT Histogram"); plt.grid(True); plt.show()

## 5️⃣  Next Steps

- 🔁 Replace in-sample with walk-forward prediction loop
- 🔬 Loop over multiple driver universes (technical, fundamental)
- 📊 Log PIT, quantile hit rate, tail stats per model run
- 🖼️ Integrate with Dash UI for visual exploration and model selection
- 🧪 Add CRPS or tail-aware scoring